# 01. 競馬データ スクレイピング 調査

In [29]:
import re
import time
import datetime

import requests
import pandas as pd
from bs4 import BeautifulSoup

## カレンダー情報のHTMLソースを取得する

In [30]:
YEAR = 2025
MONTH = 5
CALENDAR_URL = f"https://race.netkeiba.com/top/calendar.html?year={YEAR}&month={MONTH}"

# netkeibaはBot対策をしており、User-Agentなどのヘッダーがないとアクセスを拒否されることがある
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(CALENDAR_URL, headers=headers)

r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定

calendar_html = r.text
# print(calendar_html)


## HTMLソースを解析して開催日を取得する

In [31]:

racedate_list = []

soup = BeautifulSoup(calendar_html, "html.parser") # BeautifulSoupでHTML解析
tabale_data = soup.find("table", class_="Calendar_Table") # 開催日が含まれるテーブルを取得
link_set = tabale_data.find_all("a") # テーブル内の全てのリンクを取得
racedate_list += [re.search(r"\d+$", link.get("href")).group() for link in link_set] # 各リンクのhref属性から、末尾の数字（開催日ID）を正規表現で抽出し、リストに追加

# 取得した開催日を表示
print("取得した開催日:")
for racedate in racedate_list:
    print(racedate)

取得した開催日:
20250503
20250504
20250510
20250511
20250517
20250518
20250524
20250525
20250531


## 開催レース一覧のHTMLソースを取得する

In [32]:
RACE_DATE = "20250503"
RACELIST_URL = f"https://db.netkeiba.com/race/list/{RACE_DATE}/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(RACELIST_URL, headers=headers)

r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定

racelist_html = r.text
# print(racelist_html)


## HTMLソースを解析して、レースIDを取得する

In [33]:
raceId_list = []

soup = BeautifulSoup(racelist_html, "html.parser")
table_data = soup.body.find_all("dl", class_="race_top_data_info fc")

raceId_list += [
    re.search(r'^/race/\d+', link.find('a').get('href')).group().split('/')[-1]
    for link in table_data
    if link.find('a').get('title')
]

# 取得したレースIDを表示
print(f"取得したレース数:{len(raceId_list)}")
print("取得したレースID:")
for raceId in raceId_list:
    print(raceId)


取得したレース数:36
取得したレースID:
202504010101
202504010102
202504010103
202504010104
202504010105
202504010106
202504010107
202504010108
202504010109
202504010110
202504010111
202504010112
202505020301
202505020302
202505020303
202505020304
202505020305
202505020306
202505020307
202505020308
202505020309
202505020310
202505020311
202505020312
202508020301
202508020302
202508020303
202508020304
202508020305
202508020306
202508020307
202508020308
202508020309
202508020310
202508020311
202508020312


## レース情報のHTMLソースを取得する

In [48]:
RACE_ID = "202410030705"
RACE_URL = f"https://db.netkeiba.com/race/{RACE_ID}/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(RACE_URL, headers=headers)

# r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定
r.encoding = 'euc_jp' 

raceinfo_html = r.text
# print(raceinfo_html)


| 日本語        | カラム名（snake_case）     | 説明                                         |
| ------------- | ------------------------- | -------------------------------------------- |
| レースID      | `race_id`                 | 一意のレース識別子                           |
| 開催日        | `race_date`               | レースの開催日（例：2024-10-05）             |
| 開催回次      | `meeting_round`           | 開催の回数・日次（例：2回東京12日目）        |
| レースタイプ  | `race_type`               | レースのカテゴリ（例：3歳オープン）          |
| ラウンド      | `race_number`             | 当日レース番号（第何レースか）               |
| レース名      | `race_name`               | レースの名称（例：日本ダービー）             |
| レースクラス  | `race_class`              | 格付け（例：G1、G2、3勝クラス、未勝利など）  |
| レース場      | `racecourse_location`     | 開催競馬場（例：東京、中山、京都）           |
| コース種別    | `course_surface`          | コースの種類（芝・ダートなど）               |
| コース向き    | `course_direction`        | コースの向き（左・右・直線など）             |
| 距離          | `race_distance`           | レース距離（m）                              |
| 天候          | `weather`                 | 天候（例：晴、曇、雨）                       |
| 芝状態        | `track_condition`         | 馬場状態（例：良、稍重、重、不良）           |
| 頭数          | `entry_count`             | 出走馬数（レースの頭数）                     |

In [50]:
# --- 共通：HTML をパース ---
soup = BeautifulSoup(raceinfo_html, "html.parser")

# --- レース情報 DataFrame を作成 ---
# # race_id
# m = re.search(r"/race/(\d{12})/|get_race_result_horse_laptime\('(\d{12})'", raceinfo_html)
# race_id = next(g for g in m.groups() if g)

# 開催日・開催回次・レースタイプ
smalltxt = soup.select_one("p.smalltxt").get_text(" ", strip=True)
# 例: '2025年6月1日 2回東京12日目 3歳オープン'
date_ja, meeting_round, race_type, *_ = smalltxt.split()
y, mth, d = map(int, re.findall(r"\d+", date_ja))
race_date = datetime.date(y, mth, d).isoformat()

# レース番号
race_number = soup.select_one("dl.racedata dt").get_text(strip=True).replace("R", "").strip()

# レース名・クラス
title_full = soup.select_one("dl.racedata h1").get_text(strip=True)
m_class = re.search(r"\(([^)]+)\)", title_full)
race_class = m_class.group(1) if m_class else ""
race_name  = re.sub(r"\([^)]*\)", "", title_full).strip()

# 開催競馬場の場所（例：東京）
m_loc = re.search(r"\d+回([^0-9]+?)\d+日", meeting_round)
racecourse_location = m_loc.group(1) if m_loc else ""

# コース情報
span_txt = soup.select_one("dl.racedata span").get_text(" ", strip=True)
course_segment = span_txt.split("/")[0].strip()

# コース表記の正規表現を改良（ダート・芝両対応、表記ゆれ対応）
cm = re.search(r"(芝|ダ|ダート)(左|右|直線)?\s*(\d+)m", course_segment)
if cm:
    # 「ダ」は「ダート」に統一
    surface = cm.group(1)
    course_surface = "ダート" if surface in ["ダ", "ダート"] else "芝"
    course_direction = cm.group(2) if cm.group(2) else ""
    race_distance = int(cm.group(3))
else:
    course_surface = ""
    course_direction = ""
    race_distance = 0

# 天候・馬場状態（芝・ダート両対応）
m_weather = re.search(r"天候\s*:\s*([^\s/]+)", span_txt)
weather = m_weather.group(1) if m_weather else ""
m_track = re.search(r"(芝|ダート)\s*:\s*([^\s/]+)", span_txt)
track_condition = m_track.group(2) if m_track else ""

# 頭数（出走馬数）を取得
race_table = soup.find("table", class_="race_table_01")
entry_count = len(race_table.find_all("tr")) - 1 if race_table else 0  # ヘッダー行を除く


# １行分の辞書を作って DataFrame に
race_info = {
    "race_id"               : RACE_ID,
    "race_date"             : race_date,
    "meeting_round"         : meeting_round,
    "race_type"             : race_type,
    "race_number"           : race_number,
    "race_name"             : race_name,
    "race_class"            : race_class,
    "racecourse_location"   : racecourse_location,
    "course_surface"        : course_surface,
    "course_direction"      : course_direction,
    "race_distance"         : race_distance,
    "weather"               : weather,
    "track_condition"       : track_condition,
    "entry_count"           : entry_count, 
}

race_df = pd.DataFrame([race_info])
pd.set_option('display.max_columns', None)

race_df


,race_id,race_date,meeting_round,race_type,race_number,race_name,race_class,racecourse_location,course_surface,course_direction,race_distance,weather,track_condition,entry_count
0,202410030705,2024-07-20,3回小倉7日目,2歳新馬,5,2歳新馬,,小倉,芝,右,1800,曇,稍重,9


| 日本語 | カラム名 | 説明                |
| --- | ------------------- | ----------------- |
| 着順  | `finish_position`   | 着順（レースでの順位）       |
| 枠番  | `frame_number`      | 枠番（ゲート位置）         |
| 馬番  | `horse_number`      | 馬番（レースでの番号）       |
| 馬名  | `horse_name`        | 馬の名前              |
| 性別  | `sex`               | 性別（牡・牝など）         |
| 年齢  | `age`               | 馬の年齢              |
| 斤量  | `weight_carried`    | 騎手が乗る際の斤量（kg）     |
| 騎手  | `jockey`            | 騎手の名前             |
| タイム | `race_time`         | レースタイム（例: 1:34.5） |
| 着差  | `margin`            | 着差（勝ち馬との差）        |
| 上り  | `last_3f`           | 最後の3ハロンのタイム（上り）   |
| 単勝  | `odds`              | 単勝オッズ             |
| 人気  | `popularity`        | 人気順位              |
| 馬体重 | `body_weight`       | 馬体重（+-含む場合も）      |
| 調教師 | `trainer`           | 調教師の名前            |
| 馬主  | `owner`             | 馬主の名前             |
| 賞金  | `prize_money`       | 獲得賞金（単位: 円）       |

In [36]:
# --- 共通：HTML をパース ---
soup = BeautifulSoup(raceinfo_html, "html.parser")

# ---  出走馬情報 DataFrame を作成 --- 
race_table = soup.find("table", class_="race_table_01")
body_rows = race_table.find_all("tr")[1:] if race_table else []

horse_records: list[dict] = []
for tr in body_rows:
    cells = [td.get_text(strip=True) for td in tr.find_all("td")]
    # 列数不足分を空文字で埋める（最大21列想定）
    cells.extend([""] * (21 - len(cells)))

    # 性齢を分割
    age_match = re.search(r"\d+", cells[4])
    age = age_match.group() if age_match else ""
    sex = cells[4].replace(age, "")

    # horse_id を馬名リンクから取得
    link = tr.find("a", href=re.compile(r"^/horse/\d+/"))
    horse_id = ""
    if link:
        m = re.search(r"/horse/(\d+)/", link["href"])
        if m:
            horse_id = m.group(1)

    rec = {
        "race_id"        : race_id,       # レース情報との紐付け用
        "finish_position": cells[0],
        "frame_number"   : cells[1],
        "horse_number"   : cells[2],
        "horse_name"     : cells[3],
        "horse_id"       : horse_id,
        "sex"            : sex,
        "age"            : age,
        "weight_carried" : cells[5],
        "jockey"         : cells[6],
        "race_time"      : cells[7],
        "margin"         : cells[8],
        "last_3f"        : cells[11],
        "odds"           : cells[12],
        "popularity"     : cells[13],
        "body_weight"    : cells[14],
        "trainer"        : cells[18],
        "owner"          : cells[19],
        "prize_money"    : cells[20],
    }
    horse_records.append(rec)

horse_df = pd.DataFrame(horse_records, columns=[
    "race_id", "finish_position", "frame_number", "horse_number", "horse_name", "horse_id", 
    "sex", "age", "weight_carried", "jockey", "race_time", "margin",
    "last_3f", "odds", "popularity", "body_weight", "trainer", "owner", "prize_money",
])

pd.set_option('display.max_columns', None)
horse_df

,race_id,finish_position,frame_number,horse_number,horse_name,horse_id,sex,age,weight_carried,jockey,race_time,margin,last_3f,odds,popularity,body_weight,trainer,owner,prize_money
0,202401020804,1,2,4,•÷•й•§•∆•£•Ґ•ј•§•д,2021102468,ћ∆,3,55,± ћоћ‘¬Ґ,1:10.1,,34.9,21.2,9,462(-2),[≈м]∞р≥јєђЌЇ,ЊЃќ”Њїї÷,385.0
1,202401020804,1,6,12,•є•¶•£°Љ•»•к•п°Љ•…,2021100415,ћ∆,3,55,…Ќ√жљ”,1:10.1,∆±√е,34.9,20.3,8,458(+2),[јЊ]µ№Ћ№«о,љ≈ћоњі њ,385.0
2,202401020804,3,5,9,•≠•Ґ•н•ф•І•н°Љ•Ѕ•І,2021105714,•ї,3,57,∆£≤ђЌ§≤р,1:10.2,•ѓ•”,36.0,16.5,6,466(-2),[јЊ]…рєђїЌѕЇ,¬зћоЊ»≤Ґ,140.0
3,202401020804,4,5,10,•≥•≥•Ј•г•у•—°Љ•Ћ•е,2021105902,ћ∆,3,55,Ћћ¬ЉЌІ∞м,1:10.2,•Ґ•њ•ё,34.4,4.5,2,466(-2),[јЊ]≤їћµљ®єІ,¬зƒЌќЉ∞м,83.0
4,202401020804,5,7,13,•Ґ•а°Љ•л•к°Љ•ў,2021105659,ћ∆,3,55,…рЋ≠,1:10.3,1/2,34.9,2.6,1,490(+8),[јЊ]ЌІ∆їєѓ…„,•§•у•Љ•л•м°Љ•Ј•у•∞,55.0
5,202401020804,6,3,5,•ї•≠•∆•§•м•Ґ,2021100226,ћ∆,3,55,ЇЎ∆£њЈ,1:10.3,•Ґ•њ•ё,35.6,23.8,10,434(-2),[јЊ]ЊЊ± іі…„,њщї≥ЄµЌќ,
6,202401020804,7,7,14,•ф•°•Ј•н•Џ•є,2021103903,•ї,3,57,ї≠≈зєољў,1:10.3,•ѕ•,35.4,10.2,4,456(+8),[јЊ]љп э≈Ў,•“•ј•Ђ°¶•÷•к°Љ•ј°Љ•Ї°¶•ж•Ћ•™•у,
7,202401020804,8,8,16,•µ•Ћ•§•Ґ°Љ•л,2021103052,ћ∆,3,55,√∞∆вЌіЉ°,1:10.4,1/2,35.7,8.1,3,448(+2),[≈м]≈ƒ¬Љєѓњќ,¬зƒЌќЉ∞м,
8,202401020804,9,2,3,•н•у•’•©•у,2021105830,ћ∆,3,55,≤£ї≥…рїЋ,1:10.5,3/4,35.0,14.3,5,434(+6),[≈м]µ„ Ё≈ƒµЃ,Ј¶≈ƒЋІѕЇ,
9,202401020804,10,4,8,•’•І•Ґ•Ґ•§•л,2021105827,ћ∆,3,53,ЊЃќ”Њ°¬ј,1:10.6,3/4,36.2,29.2,11,404(+6),[јЊ]≤їћµљ®єІ,•≠•г•н•√•»•’•°°Љ•а,


## HTMLソースを解析して、馬IDを取得する

In [37]:
horseId_list = []

soup = BeautifulSoup(raceinfo_html, "html.parser")

# 出馬表のテーブル内だけから馬リンクを探す
horse_links = soup.select("table.race_table_01 a[href^='/horse/']")

# 正規表現で ID 部分だけ取り出す
horseId_list = [
    re.search(r"/horse/(\d+)/", a["href"]).group(1)
    for a in horse_links
]

# 重複を除去（同じ馬が複数回リンクされているケースに備えて）
horseId_list = list(dict.fromkeys(horseId_list))

# 取得した馬IDを表示
print(f"取得した馬数:{len(horseId_list)}")
print("取得した馬ID:")
for horseId in horseId_list:
    print(horseId)


取得した馬数:16
取得した馬ID:
2021102468
2021100415
2021105714
2021105902
2021105659
2021100226
2021103903
2021103052
2021105830
2021105827
2021102598
2021101921
2021103004
2021101402
2021100146
2021104033


## 馬情報のHTMLソースを取得する

In [38]:
HORSE_ID = "2022105102"
HORSE_URL = f"https://db.netkeiba.com/horse/{HORSE_ID}/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(HORSE_URL, headers=headers)

r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定

horseinfo_html = r.text
# print(horseinfo_html)


| 日本語        | カラム名（snake\_case）       | 説明             |
| ---------- | ----------------------- | -------------- |
| 馬ID        | `horse_id`              | 馬の一意識別子（主キー）   |
| 馬名         | `horse_name`            | 馬の名前           |
| 性別         | `sex`                   | 性別（'牡'／'牝'）    |
| 生年         | `birth_year`            | 誕生年（YYYY形式）    |
| 毛色         | `color`                 | 毛色（例：鹿毛、栗毛など）  |
| 調教師        | `trainer`               | 調教師の名前         |
| トレセン       | `training_center`       | 所属トレーニングセンター   |
| 馬主         | `owner`                 | 馬主（所有者）の名前     |
| 生産者        | `breeder`               | 生産牧場・生産者名      |
| 産地         | `birth_place`           | 生産地（都道府県など）    |
| 獲得賞金（中央）   | `earnings_central`      | 中央競馬での獲得賞金（通算） |
| 獲得賞金（地方）   | `earnings_local`        | 地方競馬での獲得賞金（通算） |
| 通算成績（出走回数） | `total_starts`          | 通算出走回数         |
| 通算成績1着     | `total_wins`            | 通算1着回数         |
| 通算成績2着     | `total_second_places`   | 通算2着回数         |
| 通算成績3着     | `total_third_places`    | 通算3着回数         |
| 父馬ID       | `sire_id`               | 父馬の馬ID         |
| 母馬ID       | `dam_id`                | 母馬の馬ID         |
| 父父馬ID      | `paternal_grandsire_id` | 父の父馬の馬ID       |
| 父母馬ID      | `paternal_granddam_id`  | 父の母馬の馬ID       |
| 母父馬ID      | `maternal_grandsire_id` | 母の父馬の馬ID       |
| 母母馬ID      | `maternal_granddam_id`  | 母の母馬の馬ID       |


In [39]:

# --- 共通：HTML をパース ---
soup = BeautifulSoup(horseinfo_html, "html.parser")

# --- horse_id を canonical リンクから取得 ---
horse_id = ""
can_link = soup.find("link", rel="canonical")
if can_link and can_link.has_attr("href"):
    href = can_link["href"]
    m = re.search(r"/horse/(\d+)/", href)
    if m:
        horse_id = m.group(1)

# --- 馬名 ---
horse_name_elem = soup.select_one(".horse_title h1")
horse_name = horse_name_elem.get_text(strip=True) if horse_name_elem else ""

# --- 性別・毛色（例："現役　牡3歳　青鹿毛"）を分割 ---
sex = ""
color = ""
txt01 = soup.find("p", class_="txt_01")
if txt01:
    txt = txt01.get_text(strip=True)
    parts = txt.split()
    # parts の例: ['現役', '牡3歳', '青鹿毛']
    if len(parts) >= 2:
        sex = parts[1][0]
    if len(parts) >= 3:
        color = parts[2]

# --- プロフィール情報を辞書に抽出 ---
profile = {}
prof_table = soup.find("table", class_="db_prof_table")
if prof_table:
    for tr in prof_table.find_all("tr"):
        th = tr.find("th").get_text(strip=True)
        td_elem = tr.find("td")
        td_text = td_elem.get_text(strip=True) if td_elem else ""
        if th == "生年月日":
            m = re.search(r"(\d{4})年", td_text)
            profile["birth_year"] = m.group(1) if m else ""
        elif th == "調教師":
            # 調教師名とトレセンを分離
            a = td_elem.find("a")
            if a:
                profile["trainer"] = a.get_text(strip=True)
            m2 = re.search(r"\((.*?)\)", td_text)
            profile["training_center"] = m2.group(1) if m2 else ""
        elif th == "馬主":
            a = td_elem.find("a")
            profile["owner"] = a.get_text(strip=True) if a else td_text
        elif th == "生産者":
            a = td_elem.find("a")
            profile["breeder"] = a.get_text(strip=True) if a else td_text
        elif th == "産地":
            profile["birth_place"] = td_text
        elif th == "獲得賞金":
            profile["earnings_central"] = td_text
        elif th == "通算成績":
            # [勝-2着-3着-その他] 形式をパース
            m_br = re.search(r"\[([0-9\-]+)\]", td_text)
            if m_br:
                br = m_br.group(1).split("-")
                profile["total_wins"] = br[0] if len(br) > 0 else ""
                profile["total_second_places"] = br[1] if len(br) > 1 else ""
                profile["total_third_places"] = br[2] if len(br) > 2 else ""
                try:
                    profile["total_starts"] = str(sum(int(x) for x in br))
                except ValueError:
                    profile["total_starts"] = ""
            else:
                profile["total_starts"] = ""
                profile["total_wins"] = ""
                profile["total_second_places"] = ""
                profile["total_third_places"] = ""

# --- 血統情報を抜き出し ---
sire_id = dam_id = ""
paternal_grandsire_id = paternal_granddam_id = ""
maternal_grandsire_id = maternal_granddam_id = ""
blood = soup.find("table", class_="blood_table")
if blood:
    rows = blood.find_all("tr")
    # 父馬ID, 父父馬ID
    tds0 = rows[0].find_all("td")
    a0 = tds0[0].find("a")
    if a0:
        sire_id = a0["href"].split("/")[-2]
    a1 = tds0[1].find("a")
    if a1:
        paternal_grandsire_id = a1["href"].split("/")[-2]
    # 父母馬ID
    a2 = rows[1].find("a")
    if a2:
        paternal_granddam_id = a2["href"].split("/")[-2]
    # 母馬ID, 母父馬ID
    tds2 = rows[2].find_all("td")
    a3 = tds2[0].find("a")
    if a3:
        dam_id = a3["href"].split("/")[-2]
    a4 = tds2[1].find("a")
    if a4:
        maternal_grandsire_id = a4["href"].split("/")[-2]
    # 母母馬ID
    a5 = rows[3].find("a")
    if a5:
        maternal_granddam_id = a5["href"].split("/")[-2]

# --- レコードをまとめて DataFrame 化 ---
record = {
    "horse_id": horse_id,
    "horse_name": horse_name,
    "sex": sex,
    "birth_year": profile.get("birth_year", ""),
    "color": color,
    "trainer": profile.get("trainer", ""),
    "training_center": profile.get("training_center", ""),
    "owner": profile.get("owner", ""),
    "breeder": profile.get("breeder", ""),
    "birth_place": profile.get("birth_place", ""),
    "earnings_central": profile.get("earnings_central", ""),
    "earnings_local": "",  # 別ソースがなければ空文字
    "total_starts": profile.get("total_starts", ""),
    "total_wins": profile.get("total_wins", ""),
    "total_second_places": profile.get("total_second_places", ""),
    "total_third_places": profile.get("total_third_places", ""),
    "sire_id": sire_id,
    "dam_id": dam_id,
    "paternal_grandsire_id": paternal_grandsire_id,
    "paternal_granddam_id": paternal_granddam_id,
    "maternal_grandsire_id": maternal_grandsire_id,
    "maternal_granddam_id": maternal_granddam_id,
}

horse_df = pd.DataFrame([record], columns=list(record.keys()))
pd.set_option("display.max_columns", None)
horse_df

,horse_id,horse_name,sex,birth_year,color,trainer,training_center,owner,breeder,birth_place,earnings_central,earnings_local,total_starts,total_wins,total_second_places,total_third_places,sire_id,dam_id,paternal_grandsire_id,paternal_granddam_id,maternal_grandsire_id,maternal_granddam_id
0,2022105102,クロワデュノール,牡,2022,青鹿毛,斉藤崇史,栗東,サンデーレーシング,ノーザンファーム,安平町,"5億3,248万円 (中央)",,5,4,1,0,2012102013,000a011523,2001103312,2005106935,000a0022ae,000a01144f


## 馬戦績のHTMLソースを取得する

In [40]:
HORSE_ID = "2022105102"
HORSE_RESULT_URL = f"https://db.netkeiba.com/horse/result/{HORSE_ID}/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(HORSE_RESULT_URL, headers=headers)

r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定

horseresult_html = r.text
# print(horseresult_html)

## 馬血統のHTMLソースを取得する

In [41]:
HORSE_ID = "2022105102"
HORSE_PED_URL = f"https://db.netkeiba.com/horse/ped/{HORSE_ID}/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
r = requests.get(HORSE_PED_URL, headers=headers)

r.encoding = r.apparent_encoding # レスポンスのエンコーディングを設定

horseped_html = r.text
# print(horseped_html)